<a href="https://colab.research.google.com/github/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation/blob/main/code_template_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

We were given InkML files, which contain metadata and a list of strokes: [(0,0),(1;0)]...  
These are on-line handwritten mathematical expressions, we'll try to recognize them via LG(Labelled Graph) as output.  
This is the sequence of actions performed:  
1) Determine possible stroke combinations.  
2) Remove impossible combinations with a classifier.  
3) Convert each combination to a symbol.  

We'll handle spatial relations later  
We'll also handle the final decision later, but we can add a grammar or language model at that point, 

# Environment setup

## Getting project files

We will be importing the project files from our github repository.

In [1]:
!git clone https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Cloning into 'Master_2_MLVC_Recognize_Handwritten_Equation'...
remote: Enumerating objects: 119843, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 119843 (delta 71), reused 95 (delta 39), pack-reused 119711
Receiving objects: 100% (119843/119843), 46.53 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Checking out files: 100% (209489/209489), done.


In [2]:
%cd Master_2_MLVC_Recognize_Handwritten_Equation/code

/content/Master_2_MLVC_Recognize_Handwritten_Equation/code


We will first show what each script does, and how to interpret their output, then we will show how to use the evaluation scripts, and finally detail how each script functions.

## Data to train our classifiers

[This](https://uncloud.univ-nantes.fr/index.php/s/OdAtErZgxKGjsNy) contains a bunch of symbols (in datasymbol_iso/) and expressions (in FullExpressions/) to help train our future classifiers.



## Showcasing parts of code

### segmenter.py explanation

The first action is to collect the possible stroke combinations, for now we'll simply take every consecutive stroke combinations.  
So if we have 4 strokes in one inkml file, 13 combinations can be done.  

Each line of the output corresponds to a hypothesis: indicating the symbol type, the symbol with index (starting with 1, but dummy value for now since we don't know what symbol the combination could be), then the symbol without index, then the supposed confidence of the model. We also have the strokes used next to these informations.

At this point, this can be optimized by already removed hypotheses take don't make sense, for instance trying to make a symbol with every single stroke is highly irregular and shouldn't happen, so we could remove that combination before calling the other scripts.

The ground truth for the segmentation is available in the original lg files, since the strokes used are listed next to each symbol.

In [5]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml -o ../data/example.lg

In [6]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml

O,hyp0,*,1.0,0
O,hyp1,*,1.0,1
O,hyp2,*,1.0,2
O,hyp3,*,1.0,3
O,hyp4,*,1.0,4
O,hyp5,*,1.0,0,1
O,hyp6,*,1.0,1,2
O,hyp7,*,1.0,2,3
O,hyp8,*,1.0,3,4
O,hyp9,*,1.0,0,1,2
O,hyp10,*,1.0,1,2,3
O,hyp11,*,1.0,2,3,4
O,hyp12,*,1.0,0,1,2,3
O,hyp13,*,1.0,1,2,3,4



As we've said earlier, we simply keep every consecutive stroke combination as our possible hypotheses.

## Use this subsection when testing stuff for now

In [46]:
#!git stash
#!git stash drop

No local changes to save
No stash entries found.


In [47]:
#!git reset --soft HEAD^ 

In [25]:
#!git pull

Already up to date.


In [40]:
#!git config --global user.email "XXX@gmail.com"
#!git config --global user.name "XXX"

In [48]:
#!git add .

In [49]:
#!git commit -m "Speed up inference for segmentSelect and slightly modified AlexNet again"

[main dc38065c7] Speed up inference for segmentSelect and slightly modified AlexNet again
 4 files changed, 12 insertions(+), 17 deletions(-)
 rewrite code/output.png (94%)
 rewrite code/resultMLP.png (98%)


In [50]:
#!git push https://hiddentoken@github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 42.12 KiB | 21.06 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git
   357bb3271..dc38065c7  main -> main


### CROHME_train_segmentSelector.py & segmentSelect.py explanation

Since we use neural networks in our overall process as our classifiers/predictors, they need to be trained beforehand. But let's first explain what we're trying to do in this part of the process:  

The script 'segmentSelect.py' takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations from the prototype file alongside the inkml stroke data to generate images, then we check whether these images make sense as a symbol, no matter the context.  
This is a classification problem with two possible outputs : Image is valid or invalid.  
We also check the confidence value of the model with a threshold in order to ignore unsure hypotheses, which should boost accuracy somewhat.

Now, for the training part, while we could simply store the weights of the models and import them, we still want to show the specifics of our training due to the characteristics of our data:

While training the model batch per batch, we make sure that each of these batches contain an representative random subset of the original data, since we have a lot less invalid images for training than valid images, while still trying to make sure that the model doesn't excessively consider valid images.  
The rest of the training logic is quite normal, the state of the model is saved whenever we achieve a new validation loss low, and we implemented early stopping to prevent overfitting.

In [13]:
!python3 CROHME_train_segmentSelector.py

cuda:0
('invalid', 'valid')
nb classes 2 , training size 12000, val size 4000, test size 4000
invalid valid valid valid
AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(96, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU()
  )
  (fc1): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in

### segmentSelect.py example

In [14]:
!python3 segmentSelect.py -o ../data/example2.lg ../data/formulaire001-equation001.inkml ../data/example.lg 

Traceback (most recent call last):
  File "segmentSelect.py", line 101, in <module>
    main()
  File "segmentSelect.py", line 87, in main
    prob = computeProbSeg(traces, h, saveimg)
  File "segmentSelect.py", line 55, in computeProbSeg
    output = torch.nn.functional.softmax(model(im_tensor),dim=1)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/Master_2_MLVC_Recognize_Handwritten_Equation/code/modules.py", line 55, in forward
    out = self.layer1(x)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py", line 139, in forward
    input = module(input)
  File "/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/loc

In [24]:
!python3 segmentSelect.py ../data/formulaire001-equation001.inkml ../data/example.lg 

O,hyp1,*,0.7486852407455444,1
O,hyp5,*,0.5107819437980652,0,1
O,hyp6,*,0.6763725876808167,1,2
O,hyp7,*,0.7221850752830505,2,3
O,hyp8,*,0.9450982809066772,3,4
O,hyp10,*,0.8999685645103455,1,2,3
O,hyp11,*,0.5407421588897705,2,3,4
O,hyp12,*,0.9287165403366089,0,1,2,3
O,hyp13,*,0.8904486894607544,1,2,3,4



This script takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations alongside the inkml data to generate images, then we check whether these make sense as a symbol, no matter the context.

For now this simply gives a fully random probability that the images are valid symbols.

Need to change two things in there, first adding a classifier to check the validity of supposed symbols,  
then change the threshold based on empirical evidence or what I feel like at the time.

### segmentReco.py explanation

In [ ]:
!chmod 777 ./symbolReco.py

In [ ]:
!python3 symbolReco.py ../data/formulaire001-equation001.inkml ../data/bidule2.lg

O,hyp0,r,0.059779757207917304,0
O,hyp0,geq,0.05030687013768514,0
O,hyp1,2,0.05648836776140208,1
O,hyp1,6,0.056168315175164935,1
O,hyp1,8,0.05960923199570466,1
O,hyp1,=,0.05475807870783242,1
O,hyp1,geq,0.056816822917386364,1
O,hyp3,f,0.0510902919404698,3
O,hyp3,n,0.0586707731068381,3
O,hyp3,u,0.053721497506933835,3
O,hyp3,3,0.055092772863820916,3
O,hyp4,b,0.05192545736402091,4
O,hyp4,leq,0.07015718890027181,4
O,hyp5,s,0.06121838413470821,0,1
O,hyp5,A,0.050391613253440264,0,1
O,hyp5,X,0.072619141223795,0,1
O,hyp6,(,0.05046390880442578,1,2
O,hyp7,p,0.06723378695795225,2,3
O,hyp7,y,0.05172590875612772,2,3
O,hyp7,1,0.05240988369270244,2,3
O,hyp7,geq,0.058408507485593536,2,3
O,hyp8,Z,0.053352030138174675,3,4
O,hyp9,h,0.051727432180693006,0,1,2
O,hyp9,C,0.05847255666598241,0,1,2
O,hyp9,(,0.05560318869409104,0,1,2
O,hyp9,geq,0.05338792763291579,0,1,2
O,hyp10,r,0.055058494864859676,1,2,3
O,hyp10,4,0.050618596960434595,1,2,3
O,hyp11,c,0.05128946473017116,2,3,4
O,hyp11,j,0.06079852247795053,2,3,4

TODOLIST: check that everything works: X  
Apply evaluation script (import relevant things here): X  
Nitpick a bunch of things:  X  
Write some things on notebook: X  
